<a href="https://colab.research.google.com/github/AndreasAvgou/Genetic-Bacteria-Identification/blob/main/Genetic_Bacteria_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings
import matplotlib.pyplot as plt
plt.style.use('ggplot')

plt.rcParams['figure.dpi'] = 300

import pickle

from imblearn.pipeline import Pipeline


from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, tree

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix,multilabel_confusion_matrix,classification_report
from sklearn.metrics import precision_recall_curve,plot_precision_recall_curve,plot_roc_curve
from sklearn.metrics import accuracy_score,f1_score

from tqdm.notebook import tqdm
from google.colab import drive

warnings.filterwarnings('ignore')

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Genetic Bacteria Identification/train_dataset.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Genetic Bacteria Identification/test_dataset.csv")
train_df.head(15)

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,197019,4.630000e-08,2.460000e-06,3.080000e-06,0.000063,0.000150,1.296740e-04,0.000110,0.000074,0.000003,...,0.000040,0.000030,0.000043,0.000055,0.000028,0.000001,0.000005,9.460000e-06,4.630000e-08,Salmonella_enterica
1,28056,-9.540000e-07,-5.540000e-06,3.080000e-06,0.000048,0.000087,3.870000e-05,0.000055,0.000041,-0.000009,...,0.000116,0.000068,0.000142,0.000125,0.000093,0.000009,0.000012,3.250000e-05,4.630000e-08,Escherichia_coli
2,8210,-9.540000e-07,1.050000e-05,1.710000e-05,-0.000004,-0.000020,7.970000e-05,-0.000030,0.000036,-0.000043,...,0.000054,0.000097,0.000064,0.000114,-0.000003,-0.000010,0.000021,4.630000e-07,-9.540000e-07,Streptococcus_pneumoniae
3,98476,4.630000e-08,1.460000e-06,8.470000e-08,0.000008,0.000030,2.970000e-05,0.000036,0.000022,0.000003,...,0.000048,0.000025,0.000039,0.000042,0.000019,0.000004,0.000002,4.460000e-06,4.630000e-08,Escherichia_coli
4,166342,-9.540000e-07,4.630000e-07,-1.290000e-05,0.000036,0.000060,-3.260000e-07,-0.000040,0.000066,-0.000033,...,0.000134,0.000087,0.000134,0.000134,0.000037,0.000010,0.000010,4.050000e-05,-9.540000e-07,Escherichia_fergusonii
5,102456,-9.540000e-07,-9.540000e-06,-4.290000e-05,-0.000114,-0.000200,-2.403260e-04,-0.000200,-0.000114,-0.000043,...,0.000914,-0.000043,-0.000086,0.000914,-0.000043,-0.000010,0.000990,-9.540000e-06,-9.540000e-07,Streptococcus_pneumoniae
6,170513,-9.540000e-07,-3.540000e-06,8.080000e-06,0.000180,0.000334,3.876740e-04,0.000319,0.000158,0.000006,...,0.000109,0.000071,0.000143,0.000169,0.000110,0.000016,0.000013,3.450000e-05,4.630000e-08,Salmonella_enterica
7,149431,-9.540000e-07,1.050000e-05,-2.290000e-05,0.000056,0.000050,3.970000e-05,0.000020,-0.000024,-0.000033,...,0.000154,0.000077,0.000064,0.000084,0.000047,0.000010,0.000021,1.050000e-05,-9.540000e-07,Escherichia_coli
8,49548,-9.540000e-07,-9.540000e-06,-4.290000e-05,-0.000114,0.009800,-2.403260e-04,-0.000200,-0.000114,-0.000043,...,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-0.000010,-9.540000e-06,-9.540000e-07,Escherichia_coli
9,148931,-9.540000e-07,-9.540000e-06,-3.990000e-05,-0.000100,-0.000175,-2.253260e-04,-0.000189,-0.000103,-0.000040,...,0.001170,0.000805,0.001184,0.001517,0.000871,0.000234,0.000310,3.104630e-04,7.050000e-06,Enterococcus_hirae


In [ ]:
print("============= TRAIN SET =============")
print("Missing Values: " ,np.sum(train_df.isnull().any(axis=1)))
print("Shape: ", train_df.shape)
print("Columns: ", len(train_df.columns))
print("Data types:")
print(train_df.dtypes)
print("============= TEST SET =============")
print("Missing Values: ",(np.sum(test_df.isnull().any(axis=1))))
print("Shape: ", test_df.shape)
print("Columns: ", len(test_df.columns))
print("Data types:")
print(test_df.dtypes)

============= TRAIN SET =============
Missing Values:  0
Shape:  (24000, 288)
Columns:  288
Data types:
row_id         int64
A0T0G0C10    float64
A0T0G1C9     float64
A0T0G2C8     float64
A0T0G3C7     float64
              ...   
A9T0G0C1     float64
A9T0G1C0     float64
A9T1G0C0     float64
A10T0G0C0    float64
target        object
Length: 288, dtype: object
============= TEST SET =============
Missing Values:  0
Shape:  (6000, 287)
Columns:  287
Data types:
row_id         int64
A0T0G0C10    float64
A0T0G1C9     float64
A0T0G2C8     float64
A0T0G3C7     float64
              ...   
A8T2G0C0     float64
A9T0G0C1     float64
A9T0G1C0     float64
A9T1G0C0     float64
A10T0G0C0    float64
Length: 287, dtype: object


In [ ]:
train_df.info()
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Columns: 288 entries, row_id to target
dtypes: float64(286), int64(1), object(1)
memory usage: 52.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Columns: 287 entries, row_id to A10T0G0C0
dtypes: float64(286), int64(1)
memory usage: 13.1 MB


In [ ]:
duplicate_train_df = train_df[train_df.duplicated()]
duplicate_train_df.shape

(0, 288)

In [ ]:
duplicate_test_df = test_df[test_df.duplicated()]
duplicate_test_df.shape

(0, 287)

In [ ]:
train_df.describe()
test_df.describe()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
count,6000.000000,6.000000e+03,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,...,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6.000000e+03,6.000000e+03,6.000000e+03
mean,100413.868833,-4.066713e-07,-0.000003,-0.000018,-0.000010,-0.000018,0.000012,0.000012,-0.000011,-0.000030,...,0.000140,0.000337,0.000241,0.000470,0.000583,0.000385,0.000070,9.038839e-05,1.020441e-04,-1.932172e-07
std,57209.410058,1.300272e-05,0.000134,0.000231,0.000432,0.000573,0.000823,0.000757,0.000417,0.000049,...,0.000753,0.001121,0.000875,0.001273,0.001549,0.001212,0.000537,5.456842e-04,5.802528e-04,2.432361e-06
min,61.000000,-9.540000e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-9.540000e-06,-9.540000e-06,-9.540000e-07
25%,51075.750000,-9.540000e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000043,-0.000086,-0.000086,-0.000043,-0.000010,-9.540000e-06,-9.540000e-06,-9.540000e-07
50%,100517.500000,-9.540000e-07,-0.000010,-0.000043,-0.000114,-0.000200,-0.000239,-0.000200,-0.000114,-0.000043,...,0.000010,0.000056,0.000044,0.000124,0.000114,0.000047,-0.000010,4.630000e-07,4.630000e-07,-9.540000e-07
75%,149465.250000,-9.540000e-07,-0.000003,-0.000013,-0.000004,-0.000018,0.000004,-0.000027,0.000006,-0.000029,...,0.000116,0.000454,0.000289,0.000684,0.000770,0.000368,0.000030,5.775000e-05,6.350000e-05,-9.540000e-07
max,199976.000000,9.990460e-04,0.009990,0.009957,0.009886,0.009800,0.019760,0.009800,0.009886,0.000957,...,0.009957,0.019914,0.019957,0.019914,0.019914,0.019957,0.009990,9.990463e-03,9.990463e-03,3.900000e-05


In [ ]:
#train_df = train_df.astype(object)
#test_df = test_df.astype(object)

In [ ]:

X = train_df.drop('target', axis = 1) # here we are dropping the Target feature as this is the target and 'X' is input features, the changes are not
# made inplace as we have not used 'inplace = True'
y = train_df['target'] # Output/Dependent variable
# Input/independent variables



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


In [ ]:
X_train.shape

(18000, 287)

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

array([[ 1.20898233, -0.01835638, -0.05501387, ..., -0.18100783,
        -0.18565679, -0.05520488],
       [-1.30541502, -0.01835638, -0.05501387, ..., -0.18100783,
        -0.18565679, -0.05520488],
       [-0.13388542, -0.01835638, -0.05501387, ..., -0.18100783,
        -0.18565679, -0.05520488],
       ...,
       [ 0.28353804, -0.01835638, -0.05501387, ..., -0.127938  ,
        -0.08846384, -0.05520488],
       [-1.04074253, -0.01835638,  0.12732627, ..., -0.11027161,
        -0.13009417, -0.05520488],
       [ 0.48628212, -0.01835638, -0.05501387, ..., -0.18100783,
        -0.18565679, -0.05520488]])

In [ ]:
#model1 = tree.DecisionTreeClassifier(max_depth=32)
#model = svm.LinearSVC(random_state=0, tol=1e-5)
model = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10,), random_state=1)


In [ ]:
#model1.fit(X_train,y_train)
#cvs1 = (np.mean(cross_val_score(model1, X_train,y_train, cv=10)))


In [ ]:
model.fit(X_train,y_train)
#cvs = (np.mean(cross_val_score(model, X_train,y_train, cv=10)))

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(10,), random_state=1)

In [ ]:
#cvs

In [ ]:
#print(model1.score(X_test,y_test))
print(model.score(X_test,y_test))

0.8605


In [ ]:
y_pred = model.predict(X_test)
print('Accuracy Score is: ', accuracy_score(y_test,y_pred))

Accuracy Score is:  0.8605


In [ ]:
cm = confusion_matrix(y_test,y_pred)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_test,y_pred))

                          precision    recall  f1-score   support

    Bacteroides_fragilis       0.90      0.89      0.89       626
    Campylobacter_jejuni       0.90      0.91      0.90       605
      Enterococcus_hirae       0.84      0.86      0.85       599
        Escherichia_coli       0.79      0.79      0.79       606
  Escherichia_fergusonii       0.79      0.82      0.80       587
   Klebsiella_pneumoniae       0.91      0.92      0.91       594
     Salmonella_enterica       0.88      0.82      0.85       599
   Staphylococcus_aureus       0.90      0.91      0.90       604
Streptococcus_pneumoniae       0.84      0.89      0.86       579
  Streptococcus_pyogenes       0.87      0.81      0.84       601

                accuracy                           0.86      6000
               macro avg       0.86      0.86      0.86      6000
            weighted avg       0.86      0.86      0.86      6000



In [ ]:
test_df

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,86283,-9.540000e-07,-9.540000e-06,-2.290000e-05,0.000086,-0.000010,-0.000010,0.000100,0.000026,-0.000033,...,0.000047,0.000174,0.000117,0.000144,0.000204,0.000167,0.000030,4.630000e-07,0.000030,-9.540000e-07
1,158141,-9.540000e-07,4.630000e-07,-2.290000e-05,-0.000024,-0.000100,-0.000070,-0.000140,-0.000034,-0.000043,...,0.000087,0.000444,0.000297,0.000264,0.000454,0.000177,0.000071,8.050000e-05,0.000081,-9.540000e-07
2,104780,-9.540000e-07,-9.540000e-06,3.710000e-05,0.000436,0.001160,0.001710,0.001230,0.000566,0.000087,...,0.000017,0.000014,0.000067,0.000084,0.000014,0.000017,0.000030,2.050000e-05,0.000021,-9.540000e-07
3,87545,-9.540000e-07,-8.540000e-06,-4.090000e-05,-0.000108,-0.000196,-0.000237,-0.000192,-0.000113,-0.000040,...,0.000305,0.000739,0.000460,0.000796,0.000859,0.000523,0.000117,1.624630e-04,0.000129,4.630000e-08
4,174958,4.630000e-08,-2.540000e-06,-9.150000e-07,0.000008,0.000040,0.000009,0.000021,0.000019,-0.000009,...,0.000028,0.000077,0.000050,0.000107,0.000112,0.000064,0.000007,6.460000e-06,0.000025,4.630000e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,110373,-9.540000e-07,-9.540000e-06,-4.290000e-05,-0.000114,-0.000200,-0.000240,-0.000190,-0.000104,-0.000043,...,0.000517,0.001154,0.000847,0.001204,0.001824,0.000927,0.000150,2.904630e-04,0.000250,9.050000e-06
5996,56577,4.630000e-08,3.460000e-06,8.470000e-08,-0.000015,0.000007,-0.000003,-0.000003,0.000003,0.000003,...,0.000013,0.000032,0.000027,0.000070,0.000078,0.000085,0.000009,1.150000e-05,0.000009,4.630000e-08
5997,74695,-9.540000e-07,4.630000e-07,-2.290000e-05,-0.000054,-0.000130,-0.000100,-0.000150,-0.000054,-0.000043,...,0.000357,0.000824,0.000627,0.000854,0.001014,0.000437,0.000190,1.804630e-04,0.000180,-9.540000e-07
5998,72935,-9.540000e-07,-9.540000e-06,-4.290000e-05,-0.000114,-0.000200,-0.000240,-0.000200,-0.000114,-0.000043,...,0.001957,0.000914,0.000957,0.000914,0.000914,0.000957,-0.000010,-9.540000e-06,0.000990,-9.540000e-07


In [ ]:
target = model.predict(test_df)
res = pd.DataFrame(y_pred) #preditcions are nothing but the final predictions of your model on input features of your new unseen test data
res.index = test_df.index
res.columns = ["target"]
res.to_csv("prediction_results.csv", index = True)      # the csv file will be saved locally on the same location where this notebook is located.

In [ ]:
res.head()

,target
0,Streptococcus_pneumoniae
1,Streptococcus_pyogenes
2,Streptococcus_pyogenes
3,Salmonella_enterica
4,Enterococcus_hirae
